In [1]:
from utils.utils import *
from scripts.config import DefaultArgsNamespace
import torch
import torch.nn as nn
import torchvision.models as models
from datautils.ems import *
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import wandb


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args = DefaultArgsNamespace()

# Access the parsed arguments
model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

dummy_input = torch.randn(5, 30, 3, 224, 224) # batch, num_frames, channels, height, width

dummy_input = dummy_input.to(args.device)

dummy_output = model(dummy_input)   

print(dummy_output.shape)




/home/cjh9fw/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/sfs/qumulo/qhome/cjh9fw/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/cjh9fw/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cjh9fw/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to 

torch.Size([5, 3])


In [2]:
train_loader, val_loader, test_loader = get_dataloaders(args)

Loaded 64 videos from /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/new_train_video_fold_1.txt
Video List: ['/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/oxygen-bvm/19-10-2023-12-50-31.avi', '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/oxygen-bvm/GX010307_clipped_with_audio_start_1400_end_1500.mp4', '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/oxygen-bvm/14-08-2023-14-12-02.avi', '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/oxygen-bvm/29-08-2023-20-35-49.avi', '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/oxygen-bvm/GX010313_clipped_with_audio_start_1140_end_1240.mp4', '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotatio

  0%|          | 0/4 [00:00<?, ?it/s]overread 8
overread 8
overread 8
overread 2
overread 4
overread 8
overread 8
error count: 268435465
error y=29 x=39
overread 8
overread 8
 (repeated 2 more times)
overread 5
overread 8
overread 8
 (repeated 2 more times)
overread 6
error count: 268435462
error y=29 x=39
overread 8
overread 8
error count: 268435467
error y=29 x=39
overread 8
overread 8
error count: 268435465
error y=29 x=39
overread 8
overread 8
error count: 268435465
error y=29 x=39
error count: 268435460
error y=29 x=39
error count: 268435471
error y=29 x=39
error count: 268435466
error y=29 x=39
error count: 268435473
error y=29 x=39
overread 8
 (repeated 2 more times)
error count: 268435459
error y=29 x=39
100%|██████████| 4/4 [00:49<00:00, 12.40s/it]


Computed clips for 64 videos: Total clips: 1836
Loaded 33 videos from /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/new_test_video_fold_1.txt
Video List: ['/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/oxygen-bvm/14-08-2023-14-13-12.avi', '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/oxygen-bvm/19-10-2023-12-51-03.avi', '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/oxygen-bvm/GX010310_clipped_with_audio_start_3680_end_4160.mp4', '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/oxygen-bvm/29-08-2023-20-33-57.avi', '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/oxygen-bvm/29-08-2023-20-46-36.avi', '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Inte

  0%|          | 0/3 [00:00<?, ?it/s]error count: 268435463
error y=29 x=39
error count: 268435463
error y=29 x=39
overread 8
overread 8
/home/cjh9fw/.local/lib/python3.8/site-packages/torchvision/io/video.py:404: UserWarning: Failed decoding frames for file /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/annotations/../videos/defib_pads_attachment/29-08-2023-20-38-51.avi
  warnings.warn(f"Failed decoding frames for file {filename}")
 33%|███▎      | 1/3 [00:21<00:42, 21.11s/it]overread 8
 (repeated 4 more times)
error count: 268435464
error y=29 x=39
overread 8
100%|██████████| 3/3 [00:28<00:00,  9.40s/it]

Computed clips for 33 videos: Total clips: 888
train dataset size:  184
val dataset size:  45
test dataset size:  45



/sfs/weka/scratch/cjh9fw/Rivanna/2024/repos/EgoExoEMS/Benchmarks/ActionRecognition/MTRSAP/datautils/video_utils.py:223: UserWarning: There aren't enough frames in the current video to get a clip for the given clip length and frames between clips. The video (and potentially others) will be skipped.
  warnings.warn(


In [4]:

# print one batch
for i, (videos, labels) in enumerate(train_loader):
    print(videos.shape, labels.shape)
    break

# # Train the model
# for epoch in range(args.learning_params["epochs"]):
#     train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, wandb_logger)
#     val_loss = validate(model, val_loader, criterion, device, wandb_logger)
#     scheduler.step()
#     print(f"Epoch: {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss}")


error count: 268435459
error y=29 x=39


torch.Size([10, 30, 3, 256, 256]) torch.Size([10])
